In [1]:
# importing required libraries
import numpy as np
import pandas as pd
import json

In [2]:
# Loading the data in dataframe
df=pd.read_csv("big_data.csv")

In [3]:
# Creating pairs of questions, answers
pairs=[]

for i in range(df.shape[0]):
  pairs.append(((df['Questions'][i]),df['Answers'][i]))

# Define your list of question-answer pairs
pairs = [(str(q), str(a)) for q, a in pairs]

In [4]:
# Create an empty list to hold filtered pairs
filtered_pairs = []

# Loop over the original pairs and filter out those with long questions or answers
for question, answer in pairs:
    if len(question) < 300 and len(answer) < 300:
        filtered_pairs.append((question, answer))

# Creating dataframe for filtered pairs
data = pd.DataFrame(filtered_pairs, columns=["Question", "Answer"])

In [5]:
data

,Question,Answer
0,Hi,Hello there. Tell me how are you feeling today?
1,Hey,Hi there. What brings you here today?
2,Is anyone there?,Hi there. How are you feeling today?
3,Hi there,Great to see you. How do you feel currently?
4,Hello,Hello there. Glad to see you're back. What's g...
...,...,...
3996,that's a good question. maybe it's not old age.,are you right-handed?
3997,are you right-handed?,yes. all my life.
3998,yes. all my life.,you're wearing out your right hand. stop using...
3999,you're wearing out your right hand. stop using...,but i do all my writing with my right hand.


In [6]:
# Preprocessing

import re
import string

def pre_process(text):
    # Removing newlines
    text=re.sub(r'\n', ' ',text)
    # Removing opening brackets
    text=re.sub('\(', '',text) 
    # Removing closing brackets
    text=re.sub(r'\)', '',text)
    # Removing commas 
    text=re.sub(r',', '',text) 
    # Removing hyphens
    text=re.sub(r'-', '',text)
    # Removing slashes
    text=re.sub(r'/', '',text)  
    # Removing digits
    text=re.sub(r'[0-9]', '',text)
    # Removing links
    text=re.sub(r'http\S+', '',text)

    # Removing punctuations
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = " ".join(text.split())
    # Converting data to lowercase
    text = text.lower()
    
    return text

In [7]:
# Apply pre-processing:
for i in range(data.shape[0]):
    data['Question'][i] = pre_process(data['Question'][i])

In [8]:
data

,Question,Answer
0,hi,Hello there. Tell me how are you feeling today?
1,hey,Hi there. What brings you here today?
2,is anyone there,Hi there. How are you feeling today?
3,hi there,Great to see you. How do you feel currently?
4,hello,Hello there. Glad to see you're back. What's g...
...,...,...
3996,thats a good question maybe its not old age,are you right-handed?
3997,are you righthanded,yes. all my life.
3998,yes all my life,you're wearing out your right hand. stop using...
3999,youre wearing out your right hand stop using i...,but i do all my writing with my right hand.


In [9]:
# Create a database of questions and answers
questions = data['Question'].values
answers = data['Answer'].values

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Preprocess the questions in the database
vectorizer = TfidfVectorizer()
question_vectors = vectorizer.fit_transform(questions)

In [11]:
# # Process a user input
# user_input_vector = vectorizer.transform([user_input])

# # Find the closest matching question in the database
# cosine_similarities = cosine_similarity(user_input_vector, question_vectors)
# closest_index = np.argmax(cosine_similarities)
# closest_question = questions[closest_index]

# # Retrieve the corresponding answer and return it as the chatbot's response
# response = answers[closest_index]
# print(response)

In [12]:
# Testing the model
import random
from tkinter import *

# Creating an interface for interacting with user
BG_GRAY="#ABB2B9"
BG_COLOR="#17202A"
TEXT_COLOR="#EAECEE"
BUTTON_COLOR="#FFFDD0"

FONT="Helvetica 14"
FONT_BOLD="Helvetica 13 bold"

class ChatbotGUI:
    # Function called once class object is created
    def __init__(self):
        self.window = Tk()
        self._setup_main_window()
    
    # Displaying the main window to user
    def _setup_main_window(self):
        # Giving basic details about window
        self.window.title("Mental Healthcare Chatbot")
        self.window.resizable(width=False,height=False)
        self.window.configure(width=600, height=700, bg=BG_COLOR)
        
        # Displaying the window with labels
        head_label=Label(self.window,bg=BG_COLOR,fg=TEXT_COLOR,text="Welcome",font=FONT_BOLD,pady=10)
        head_label.place(relwidth=1)
        
        line=Label(self.window,width=450,bg=BG_GRAY)
        line.place(relwidth=1,rely=0.07,relheight=0.012)
        
        # Giving text and window width and height
        self.text_widget=Text(self.window,width=20,height=2,bg=BG_COLOR,fg=TEXT_COLOR,font=FONT,padx=5,pady=5)
        self.text_widget.place(relwidth=1,rely=0.08,relheight=0.745)
        self.text_widget.configure(cursor="arrow",state=DISABLED)
        
        # Defining scrollbar for the interface
        scrollbar=Scrollbar(self.text_widget)
        scrollbar.place(relheight=1,relx=0.974)
        scrollbar.configure(command=self.text_widget.yview)
        
        bottom_label=Label(self.window,bg=BG_GRAY,height=60)
        bottom_label.place(relwidth=1,rely=0.825)
        
        # Path for messages
        self.msg_entry=Entry(bottom_label,bg="#2C3E50",fg=TEXT_COLOR,font=FONT)
        self.msg_entry.place(relwidth=0.74,relheight=0.06,rely=0.008,relx=0.011)
        self.msg_entry.focus()
        self.msg_entry.bind("<Return>",self._on_entry_pressed)
        
        send_button=Button(bottom_label,text="Enter",bg=BUTTON_COLOR,font=FONT_BOLD,width=20, command=lambda:self._on_entry_pressed(None))
        send_button.place(relx=0.77,rely=0.008,relheight=0.06,relwidth=0.22)
        
    # Message entry function
    def _on_entry_pressed(self,entry):
        msg=self.msg_entry.get()
        self._insert_message(msg,"You")
            
     # Add message in interface   
    def _insert_message(self,msg,sender):
        # If user didn't wrote anything return
        if not msg:
            return
        
        self.msg_entry.delete(0,END)
        msg1=f"{sender}: {msg}\n\n"
        self.text_widget.configure(state=NORMAL)
        self.text_widget.insert(END,msg1)
        self.text_widget.configure(state=DISABLED)
        
        # Process a user input
        user_input_vector = vectorizer.transform([msg])

        # Find the closest matching question in the database
        cosine_similarities = cosine_similarity(user_input_vector, question_vectors)
        closest_index = np.argmax(cosine_similarities)
        closest_question = questions[closest_index]

        # Retrieve the corresponding answer and return it as the chatbot's response
        response = answers[closest_index]
        
        # Display the output got from model
        bot_name="Saarthi"
        msg2=f"{bot_name}: {response}\n\n"
        self.text_widget.configure(state=NORMAL)
        self.text_widget.insert(END,msg2)
        self.text_widget.configure(state=DISABLED)
        
        self.text_widget.see(END)
        
    # Repeat the process again  
    def start(self):
        self.window.mainloop() # start the GUI

# Main function
if __name__ == "__main__":
    gui = ChatbotGUI()
    gui.start()